Instalar bibliotecas

In [1]:
# pip install google-cloud-bigquery

Importar bibliotecas

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

Conectar com o GCP

In [3]:
credentials = service_account.Credentials.from_service_account_file("ml-na-saude-ed1fc3c1a83e.json")

In [4]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

Verificar se a tabela manutencao_dw existe
- Se não existe:
    - Criar tabela
    - criar variável proximo_ano = "2020" e proximo_mes =  "01"
- Se já existe:
    - Fazer query e verificar valores de proximo_ano e proximo_mes

In [5]:
# Obter o database
dataset_sisab = client.dataset("indicadores_sisab")

In [6]:
try:
    dataset = client.get_dataset(dataset_sisab)
    print(f"O conjunto de dados {dataset_sisab} já foi criado no GCP.")
except Exception:
    client.create_dataset(dataset_sisab)
    print(f"O conjunto de dados {dataset_sisab} foi criado no GCP.")

O conjunto de dados ml-na-saude.indicadores_sisab já foi criado no GCP.


In [6]:
if client.get_dataset(dataset_sisab):
    print(f"O conjunto de dados {dataset_sisab} já foi criado no GCP.")
else:
    client.create_dataset(dataset_sisab)
    print(f"O conjunto de dados {dataset_sisab} foi criado no GCP.")

O conjunto de dados ml-na-saude.indicadores_sisab já foi criado no GCP.


In [7]:
# Tabela e schema da tabela dim_indicador
table_dim_indicador = dataset_sisab.table("dim_indicador")

schema_dim_indicador = [
    bigquery.SchemaField("sk_indicador", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("nm_indicador", "STRING"),
    bigquery.SchemaField("de_indicador", "STRING")
]

In [8]:
# Tabela e schema da tabela dim_equipe
table_dim_equipe = dataset_sisab.table("dim_equipe")

schema_dim_equipe = [
    bigquery.SchemaField("sk_equipe", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("nm_equipe", "STRING"),
    bigquery.SchemaField("de_equipe", "STRING")
]

In [9]:
# Tabela e schema da tabela dim_periodo
table_dim_periodo = dataset_sisab.table("dim_periodo")

schema_dim_periodo = [
    bigquery.SchemaField("sk_periodo", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("ano", "INTEGER"),
    bigquery.SchemaField("quadrimestre", "INTEGER")
]

In [11]:
# Tabela e schema da tabela dim_localidade
table_dim_localidade = dataset_sisab.table("dim_localidade")

schema_dim_localidade = [
    bigquery.SchemaField("sk_localidade", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("uf", "STRING"),
    bigquery.SchemaField("municipio", "STRING")
]

In [12]:
# Tabela e schema da tabela fato_sisab
table_fato_sisab = dataset_sisab.table("fato_sisab")

schema_fato_sisab = [
    bigquery.SchemaField("sk_indicador", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("sk_equipe", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("sk_periodo", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("sk_localidade", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("numerador", "INTEGER"),
    bigquery.SchemaField("denominador_utilizado", "INTEGER"),
    bigquery.SchemaField("percentual_quadrimestre", "INTEGER"),
    bigquery.SchemaField("denominador_identificado", "INTEGER"),
    bigquery.SchemaField("denominador_estimado", "INTEGER"),
    bigquery.SchemaField("cadastro", "INTEGER"),
    bigquery.SchemaField("base_externa", "INTEGER"),
    bigquery.SchemaField("percentual", "INTEGER"),
    bigquery.SchemaField("populacao", "INTEGER")
]

In [13]:
tabelas = {
    table_dim_indicador:schema_dim_indicador,
    table_dim_equipe:schema_dim_equipe,
    table_dim_periodo:schema_dim_periodo,
    table_dim_localidade:schema_dim_localidade,
    table_fato_sisab:schema_fato_sisab
}

In [14]:
for tabela_ref, schema in tabelas.items():
    try:
        client.get_table(tabela_ref, timeout=30)
        print(f"A tabela {tabela_ref} já existe!")
    except:
        client.create_table(bigquery.Table(tabela_ref, schema=schema))
        print(f"A tabela {tabela_ref} foi criada.")

A tabela ml-na-saude.indicadores_sisab.dim_indicador já existe!
A tabela ml-na-saude.indicadores_sisab.dim_equipe já existe!
A tabela ml-na-saude.indicadores_sisab.dim_periodo já existe!
A tabela ml-na-saude.indicadores_sisab.dim_localidade já existe!
A tabela ml-na-saude.indicadores_sisab.fato_sisab já existe!


Verificar se as tabelas do DW já existem:
- Se não existem:
    - Criar tabelas

Baixar arquivos a partir do proximo_ano e proximo_mes até a última data disponibilizada pela fonte